In [16]:
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-1106",
)
chat.save("model.json")

In [17]:
from langchain.llms.loading import load_llm

load_chat = load_llm("model.json")
print(load_chat)

OpenAIChat
Params: {'model_name': 'gpt-3.5-turbo-1106', 'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


/Users/min/Documents/workspace/nomad-challenge/fullstack-gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/min/Documents/workspace/nomad-challenge/fullstack-gpt/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
